In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib

In [2]:

df = pd.read_csv(r"C:\Users\priya\OneDrive\Desktop\clean_insurance_risk_claims_dataset 1.csv")

In [3]:
df

,Policy_ID,Customer_Age,Gender,Policy_Type,Annual_Income,Claim_History,Fraudulent_Claim,Premium_Amount,Claim_Amount,Risk_Score
0,POL100001,65,Other,Home,456149,2,0,44166,39636.66,Low
1,POL100002,73,Other,Auto,436828,1,0,47534,25199.46,Medium
2,POL100003,55,Male,Travel,1854954,1,0,10504,17459.27,High
3,POL100004,48,Other,Home,525041,2,0,37600,30658.92,Low
4,POL100005,61,Other,Life,1792913,5,0,51598,92316.55,High
...,...,...,...,...,...,...,...,...,...,...
495,POL100496,40,Other,Life,1020175,4,0,27954,37181.77,Low
496,POL100497,75,Other,Travel,1160135,3,1,29044,55097.75,Low
497,POL100498,43,Other,Health,702677,1,0,87324,137576.79,High
498,POL100499,22,Other,Auto,1234918,3,1,77459,57510.31,Medium


In [4]:

df['Annual_Income'].fillna(df['Annual_Income'].median(), inplace=True)
df['Customer_Age'].fillna(df['Customer_Age'].median(), inplace=True)
df['Policy_Type'].fillna(df['Policy_Type'].mode()[0], inplace=True)
df['Gender'].fillna(df['Gender'].mode()[0], inplace=True)

C:\Users\priya\AppData\Local\Temp\ipykernel_22952\128929912.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Annual_Income'].fillna(df['Annual_Income'].median(), inplace=True)
C:\Users\priya\AppData\Local\Temp\ipykernel_22952\128929912.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves

In [5]:
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[(df[column] >= lower) & (df[column] <= upper)]

df = remove_outliers(df, "Claim_Amount")

In [6]:
plt.figure(figsize=(6, 4))
sns.countplot(data=df, x='Fraudulent_Claim', palette='Set2')
plt.title("Fraudulent vs Genuine Claims")
plt.xlabel("Claim Type (0 = Genuine, 1 = Fraudulent)")
plt.ylabel("Count")
plt.show()



NameError: name 'plt' is not defined

In [7]:
for col in num_cols:
    plt.figure(figsize=(6, 4))
    sns.boxplot(data=df, x=col)
    plt.title(f"Boxplot of {col}")
    plt.show()

NameError: name 'num_cols' is not defined

In [8]:
cat_cols = ["Gender", "Policy_Type", "Risk_Score"]
for col in cat_cols:
    plt.figure(figsize=(6, 4))
    sns.countplot(data=df, x=col, palette="pastel")
    plt.title(f"Count of {col}")
    plt.xticks(rotation=30)
    plt.show()

NameError: name 'plt' is not defined

In [9]:
plt.figure(figsize=(10, 6))
corr = df.select_dtypes(include='number').corr()
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

NameError: name 'plt' is not defined

In [10]:

risk_map = {"Low": 0, "Medium": 1, "High": 2}
df["Risk_Score"] = df["Risk_Score"].map(risk_map)

In [11]:

df = pd.get_dummies(df, columns=["Gender", "Policy_Type"])

In [12]:

scaler = MinMaxScaler()
num_cols = ["Annual_Income", "Claim_Amount", "Premium_Amount"]
df[num_cols] = scaler.fit_transform(df[num_cols])


In [13]:

X = df.drop(columns=["Policy_ID", "Fraudulent_Claim"])
y = df["Fraudulent_Claim"]

In [14]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
model = RandomForestClassifier(n_estimators=150, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=150, random_state=42)

In [16]:

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Model Accuracy: {accuracy * 100:.2f}%\n")

✅ Model Accuracy: 96.00%



In [17]:
RandomForestClassifier(class_weight='balanced')


RandomForestClassifier(class_weight='balanced')

In [18]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)


In [19]:

joblib.dump(model, "random_forest_model.pkl")
joblib.dump(X.columns.tolist(), "feature_columns.pkl")

['feature_columns.pkl']

In [20]:
sample_input = X_test.iloc[0:1]
sample_pred = model.predict(sample_input)[0]
print(f"\n🔮 Sample Prediction: {'Fraudulent' if sample_pred == 1 else 'Genuine'}")



🔮 Sample Prediction: Genuine
